In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [5]:
zip_path = '/content/drive/MyDrive/Crop Disease.zip'

In [6]:
import zipfile

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')  # Extract to /content directory in Colab

In [7]:
# Path to the extracted "data" directory
data_path = '/content/PlantVillage'

In [9]:
# Define image dimensions
img_width, img_height = 128, 128

# Define the dataset directory
dataset_dir = data_path

# Define number of classes (disease categories)
num_classes = len(os.listdir(dataset_dir))

# Create ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Generate batches of augmented data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Save the model
model.save('crop_disease_model.h5')

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
Epoch 1/10
516/516 [==============================] - 708s 1s/step - loss: 1.1912 - accuracy: 0.6133 - val_loss: 0.6371 - val_accuracy: 0.7825
Epoch 2/10
516/516 [==============================] - 698s 1s/step - loss: 0.5712 - accuracy: 0.8054 - val_loss: 0.5020 - val_accuracy: 0.8311
Epoch 3/10
516/516 [==============================] - 711s 1s/step - loss: 0.4271 - accuracy: 0.8566 - val_loss: 0.4227 - val_accuracy: 0.8511
Epoch 4/10
516/516 [==============================] - 693s 1s/step - loss: 0.3574 - accuracy: 0.8798 - val_loss: 0.3086 - val_accuracy: 0.8948
Epoch 5/10
516/516 [==============================] - 698s 1s/step - loss: 0.2820 - accuracy: 0.9018 - val_loss: 0.3770 - val_accuracy: 0.8704
Epoch 6/10
516/516 [==============================] - 692s 1s/step - loss: 0.2337 - accuracy: 0.9209 - val_loss: 0.2863 - val_accuracy: 0.9041
Epoch 7/10
516/516 [==============================] - 6

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
